In [2]:
import sys

sys.path.append("../src/")

In [1]:
import pathlib

import numpy as np
import pandas as pd

In [23]:
input_dir = pathlib.Path("../data/raw")

train = pd.read_csv(input_dir / "train.csv")
train_labels = pd.read_csv(input_dir / "train_labels.csv")
test = pd.read_csv(input_dir / "test.csv")
sample_submission = pd.read_csv(input_dir / "sample_submission.csv")

In [24]:
dfs = {
    "train": train,
    "train_labels": train_labels,
    "test": test,
    "sample_submission": sample_submission,
}


for name, df in dfs.items():
    print("\n\n===", name, "===")
    print(df.shape)
    display(df.head())
    print(df.info())



=== train ===
(13174211, 20)


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13174211 entries, 0 to 13174210
Data columns (total 20 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      int64  
 1   index           int64  
 2   elapsed_time    int64  
 3   event_name      object 
 4   name            object 
 5   level           int64  
 6   page            float64
 7   room_coor_x     float64
 8   room_coor_y     float64
 9   screen_coor_x   float64
 10  screen_coor_y   float64
 11  hover_duration  float64
 12  text            object 
 13  fqid            object 
 14  room_fqid       object 
 15  text_fqid       object 
 16  fullscreen      float64
 17  hq              float64
 18  music           float64
 19  level_group     object 
dtypes: float64(9), int64(4), object(7)
memory usage: 2.0+ GB
None


=== train_labels ===
(212022, 2)


,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090314121766812_q1,1
3,20090314363702160_q1,1
4,20090314441803444_q1,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212022 entries, 0 to 212021
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   session_id  212022 non-null  object
 1   correct     212022 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ MB
None


=== test ===
(3728, 21)


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,session_level
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-413.991405,75.685314,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4,20090109393214576_0-4
1,20090109393214576,1,1965,person_click,basic,0,NaN,-105.991405,-63.314686,688.0,...,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,20090109393214576_0-4
2,20090109393214576,2,3614,person_click,basic,0,NaN,-418.991405,47.685314,375.0,...,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,20090109393214576_0-4
3,20090109393214576,3,5330,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,20090109393214576_0-4
4,20090109393214576,4,6397,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4,20090109393214576_0-4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3728 entries, 0 to 3727
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   session_id      3728 non-null   int64  
 1   index           3728 non-null   int64  
 2   elapsed_time    3728 non-null   int64  
 3   event_name      3728 non-null   object 
 4   name            3728 non-null   object 
 5   level           3728 non-null   int64  
 6   page            153 non-null    float64
 7   room_coor_x     3366 non-null   float64
 8   room_coor_y     3366 non-null   float64
 9   screen_coor_x   3366 non-null   float64
 10  screen_coor_y   3366 non-null   float64
 11  hover_duration  353 non-null    float64
 12  text            1162 non-null   object 
 13  fqid            2505 non-null   object 
 14  room_fqid       3728 non-null   object 
 15  text_fqid       1162 non-null   object 
 16  fullscreen      0 non-null      float64
 17  hq              0 non-null      f

,session_id,correct,session_level
0,20090109393214576_q1,0,20090109393214576_0-4
1,20090312143683264_q1,0,20090312143683264_0-4
2,20090312331414616_q1,0,20090312331414616_0-4
3,20090109393214576_q2,0,20090109393214576_0-4
4,20090312143683264_q2,0,20090312143683264_0-4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   session_id     54 non-null     object
 1   correct        54 non-null     int64 
 2   session_level  54 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.4+ KB
None


## Todo memo

- 何人のユニークユーザのデータが使える？
- session_id+q_idでいくつのユニークデータがある？
- trainとtestでsession_idの重複はないよね？（時系列データですし）
- カテゴリと数字データは分けて分析したいね
  - train, testでカテゴリデータは網羅されていると思ってよい？ない場合はnullで補完する必要がある
- 予測対象のユーザはユニーク？
- 特徴量はグループごとに作成し、0-4の特徴量でq_id in(0,1,2,3,4)を予測する？

## カラムの意味

- session_id - イベントが発生したセッションのID
- index - セッション内でのイベントのインデックス
- elapsed_time - セッションの開始からイベントの記録までに経過した時間（ミリ秒単位）
- event_name - イベントの種類の名前
- name - イベント名（例えば、ノートブックをクリックしたのが開くか閉じるかを識別する）
- level - イベントが発生したゲームのレベル（0から22）
- page - イベントのページ番号（ノートブック関連のイベントのみ）
- room_coor_x - クリックした場所のゲーム内の部屋に対する座標（クリックイベントのみ）
- room_coor_y - クリックした場所のゲーム内の部屋に対する座標（クリックイベントのみ）
- screen_coor_x - クリックした場所のプレイヤーの画面に対する座標（クリックイベントのみ）
- screen_coor_y - クリックした場所のプレイヤーの画面に対する座標（クリックイベントのみ）
- hover_duration - ホバーイベントが発生した時間（ミリ秒単位）（ホバーイベントのみ）
- text - プレイヤーがこのイベントで見たテキスト
- fqid - イベントの完全修飾ID
- room_fqid - イベントが発生した部屋の完全修飾ID
- text_fqid - イベントに関連するテキストの完全修飾ID
- fullscreen - プレイヤーがフルスクリーンモードかどうか
- hq - ゲームが高品質モードかどうか
- music - ゲームの音楽がオンかオフか
- level_group - この行が属するレベルグループ（および質問グループ）（0-4、5-12、13-22）

In [30]:
train_labels = train_labels.assign(
    qestion_id=train_labels["session_id"].apply(lambda x: x.split('_')[1]),
    session_id=train_labels["session_id"].apply(lambda x: x.split('_')[0])
)

## Num Unique session_id

In [31]:
for name, df in dfs.items():
    print("\n\n===", name, "===")
    display(df.session_id.nunique())



=== train ===


11779



=== train_labels ===


212022



=== test ===


3



=== sample_submission ===


54